In [1]:
import importlib

import numpy as np
import torch
from tqdm import tqdm

import train_classifier

In [2]:
importlib.reload(train_classifier)

<module 'train_classifier' from '/home/aplesner/code/Ensemble-everything-everywhere-recreating-results/train_classifier.py'>

In [3]:
net = train_classifier.get_network()

In [4]:
trainloader, testloader = train_classifier.get_data_loaders()

Files already downloaded and verified
Files already downloaded and verified


In [5]:
criterion, optimizer = train_classifier.get_criterion_and_optimizer(net, lr=0.001)

In [82]:
train_classifier.train_and_eval_classifier(net=net, trainloader=trainloader, testloader=testloader, criterion=criterion, optimizer=optimizer, n_epochs=10)

Epoch 1 of 2 started


Evaluating...: 100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:10<00:00,  3.98it/s, loss=2.6609, acc=79.46%]


Epoch 1 of 2 finished. Loss: 26608.9458, Accuracies:
[0.2573 0.3923 0.458  0.5009 0.5268 0.5885 0.5137 0.6342 0.5781 0.6315
 0.6024 0.5952 0.6983 0.6712 0.6578 0.6805 0.6968 0.6128 0.7219 0.7181
 0.7096 0.6807 0.6787 0.7668 0.7042 0.7104 0.7218 0.7136 0.7063 0.7287
 0.7719 0.7137 0.7818 0.7053 0.7929 0.7746 0.7968 0.7345 0.7258 0.7586
 0.8117 0.7065 0.7864 0.8045 0.7977 0.8253 0.8321 0.7797 0.8243 0.7831
 0.7791 0.7527 0.7946]

Epoch 2 of 2 started


Evaluating...: 100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:11<00:00,  3.44it/s, loss=2.9448, acc=80.83%]

Epoch 2 of 2 finished. Loss: 29448.0956, Accuracies:
[0.2338 0.3466 0.5985 0.5464 0.5091 0.6175 0.5931 0.6018 0.66   0.6215
 0.6556 0.667  0.6236 0.7082 0.6986 0.7085 0.6737 0.6876 0.6933 0.7116
 0.7536 0.7141 0.7329 0.7488 0.7449 0.7519 0.7595 0.733  0.7376 0.7519
 0.7389 0.7371 0.7677 0.7735 0.7637 0.7962 0.7469 0.7647 0.8209 0.8281
 0.7864 0.8274 0.7707 0.8213 0.8415 0.8237 0.8352 0.8249 0.8553 0.8141
 0.8062 0.7755 0.8083]

Finished Training


In [28]:
inputs, labels = next(iter(testloader))

In [7]:
images.max(), images.min()

(tensor(2.5703), tensor(-2.1179))

In [8]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 763419 KiB | 763419 KiB | 763419 KiB |      0 B   |
|       from large pool | 678872 KiB | 678872 KiB | 678872 KiB |      0 B   |
|       from small pool |  84547 KiB |  84547 KiB |  84547 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 763419 KiB | 763419 KiB | 763419 KiB |      0 B   |
|       from large pool | 678872 KiB | 678872 KiB | 678872 KiB |

In [10]:
perturbed_inputs = train_classifier.fgsm_attacks(net=net, inputs=images, labels=labels, criterion=criterion, epsilon=8/255)

OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 8.81 MiB is free. Including non-PyTorch memory, this process has 23.68 GiB memory in use. Of the allocated memory 23.10 GiB is allocated by PyTorch, and 254.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [32]:
# inputs = images.clone().detach()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)
epsilon = 8/255

net.eval()
# net = net.to(device="cpu")
inputs = inputs.to(device)
labels = labels.to(device, non_blocking=True) 

inputs.requires_grad = True
outputs = net(inputs)


perturbed_inputs = []
for i in tqdm(range(outputs.shape[1])):
    loss = criterion(outputs[:, i, :], labels)
    net.zero_grad()
    if i < outputs.shape[1] - 1:
        loss.backward(retain_graph=True)
    else:
        loss.backward()
    with torch.no_grad():
        inputs_grad = inputs.grad
        perturbed_inputs.append(inputs + epsilon * inputs_grad.sign())


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:01<00:00, 49.66it/s]
